# 01 – Data Processing (Setup Instructions)

**Before running this notebook:**

1. Make sure the folder **`HODL Final Project`** (with `data_raw`, `data_clean`, `outputs`, etc.) is in **your** Google Drive under:  
   `My Drive/HODL Final Project`

   - If this was shared with you, go to **"Shared with me"** in Drive,  
     right-click **`HODL Final Project` → "Add shortcut to Drive" → My Drive**.

2. If your folder is in a different location or has a different name,  
   **edit the `BASE_PATH` variable in the next code cell**.

3. Then go to **Runtime → Run all** and authorize Drive access when prompted.


**Purpose: Load + clean data, establish ML baseline, export parquet file for downstream modeling**

In [1]:
# Install required packages
!pip install --quiet yfinance vaderSentiment xgboost pyarrow

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import pickle
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Stock data
import yfinance as yf

# NLP
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

# ML
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Evaluation
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Utilities
from scipy.sparse import hstack
import os
import re

# Change this if your folder is in a different place
BASE_PATH = '/content/drive/MyDrive/HODL Final Project'

DATA_RAW = f'{BASE_PATH}/data_raw'
DATA_CLEAN = f'{BASE_PATH}/data_clean'
OUTPUTS = f'{BASE_PATH}/outputs'

for path in [DATA_RAW, DATA_CLEAN, OUTPUTS]:
    os.makedirs(path, exist_ok=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.9 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
# CONFIG
# Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Stocks with sufficient tweets for modeling (>= 1000 tweets)
STOCKS = ['TSLA', 'TSM', 'AAPL', 'AMZN', 'MSFT', 'PG', 'NIO', 'META', 'AMD', 'NFLX', 'GOOG']

# Minimum tweet length to keep
MIN_TWEET_LENGTH = 20

**1.1 Load + Clean Data**

In [3]:
raw_path = f'{DATA_RAW}/stock_tweets.csv'
df_raw = pd.read_csv(raw_path)

print(f"Raw data shape: {df_raw.shape}")
print(f"Columns: {list(df_raw.columns)}")

Raw data shape: (80793, 4)
Columns: ['Date', 'Tweet', 'Stock Name', 'Company Name']


In [4]:
df_raw.head()

,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."


In [5]:
# Check universe
df_raw['Stock Name'].unique()

array(['TSLA', 'MSFT', 'PG', 'META', 'AMZN', 'GOOG', 'AMD', 'AAPL',
       'NFLX', 'TSM', 'KO', 'F', 'COST', 'DIS', 'VZ', 'CRM', 'INTC', 'BA',
       'BX', 'NOC', 'PYPL', 'ENPH', 'NIO', 'ZS', 'XPEV'], dtype=object)

In [6]:
# This will be our analysis tickers
counts = df_raw['Stock Name'].value_counts()
tickers_with_1000 = counts[counts >= 1000].index.tolist()
tickers_with_1000

['TSLA',
 'TSM',
 'AAPL',
 'AMZN',
 'MSFT',
 'PG',
 'NIO',
 'META',
 'AMD',
 'NFLX',
 'GOOG']

In [7]:
# Parse dates
df_raw['Date'] = pd.to_datetime(df_raw['Date'], utc=True)
df_raw['date_only'] = df_raw['Date'].dt.date

print(f"\nDate range: {df_raw['date_only'].min()} to {df_raw['date_only'].max()}")

# Filter to stocks with sufficient data
df = df_raw[df_raw['Stock Name'].isin(STOCKS)].copy()
print(f"\nAfter filtering to {len(STOCKS)} stocks: {df.shape[0]} tweets")

# Basic text cleaning
def clean_tweet(text):
    """Light cleaning: remove URLs, normalize whitespace"""
    if pd.isna(text):
        return ""
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove @mentions (optional, but we'll keep for now as they may carry signal)
    # text = re.sub(r'@\w+', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['tweet_clean'] = df['Tweet'].apply(clean_tweet)
df['tweet_len'] = df['tweet_clean'].str.len()

# Remove very short tweets
before_len = len(df)
df = df[df['tweet_len'] >= MIN_TWEET_LENGTH].copy()
print(f"Removed {before_len - len(df)} tweets shorter than {MIN_TWEET_LENGTH} chars")
print(f"Remaining tweets: {len(df)}")

# Summary statistics
print("\n" + "=" * 50)
print("CLEANED DATA SUMMARY")
print("=" * 50)
print(f"Total tweets: {len(df)}")
print(f"Date range: {df['date_only'].min()} to {df['date_only'].max()}")
print(f"Unique dates: {df['date_only'].nunique()}")
print(f"\nTweets per stock:")
print(df['Stock Name'].value_counts())
print(f"\nTweet length stats:")
print(df['tweet_len'].describe())

# Save cleaned tweets
df.to_csv(f'{DATA_RAW}/tweets_cleaned.csv', index=False)
print(f"\nSaved cleaned tweets to {DATA_RAW}/tweets_cleaned.csv")


Date range: 2021-09-30 to 2022-09-29

After filtering to 11 stocks: 76796 tweets
Removed 842 tweets shorter than 20 chars
Remaining tweets: 75954

CLEANED DATA SUMMARY
Total tweets: 75954
Date range: 2021-09-30 to 2022-09-29
Unique dates: 365

Tweets per stock:
Stock Name
TSLA    37200
TSM     10705
AAPL     5017
PG       4064
MSFT     4064
AMZN     4064
NIO      2972
META     2717
AMD      2208
NFLX     1684
GOOG     1259
Name: count, dtype: int64

Tweet length stats:
count    75954.000000
mean       150.836835
std         86.125547
min         20.000000
25%         72.000000
50%        136.000000
75%        239.000000
max        443.000000
Name: tweet_len, dtype: float64

Saved cleaned tweets to /content/drive/MyDrive/HODL Final Project/data_raw/tweets_cleaned.csv


**1.2 Pull stock prices, create labels**

In [8]:
# Define date range (with small buffer for next-day returns)
start_date = '2021-09-25'  # A few days before first tweet
end_date = '2022-10-05'    # A few days after last tweet

print("Pulling stock prices from Yahoo Finance...")
print(f"Date range: {start_date} to {end_date}")

# Pull prices for all stocks
price_data = []

for ticker in STOCKS:
    print(f"  Fetching {ticker}...", end=" ")
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)

        if len(hist) == 0:
            print("NO DATA")
            continue

        hist = hist.reset_index()
        hist['ticker'] = ticker
        hist['Date'] = pd.to_datetime(hist['Date']).dt.tz_localize(None)
        hist['date_only'] = hist['Date'].dt.date

        # Calculate daily return
        hist['close'] = hist['Close']
        hist['return'] = hist['close'].pct_change()

        # Next-day return (shift -1 so today's row has tomorrow's return)
        hist['next_day_return'] = hist['return'].shift(-1)

        # Binary label: 1 if next day return > 0
        hist['label'] = (hist['next_day_return'] > 0).astype(int)

        price_data.append(hist[['date_only', 'ticker', 'close', 'return', 'next_day_return', 'label']])
        print(f"{len(hist)} days")

    except Exception as e:
        print(f"ERROR: {e}")

# Combine all price data
df_prices = pd.concat(price_data, ignore_index=True)
print(f"\nTotal price records: {len(df_prices)}")
print(f"Trading days per stock: ~{len(df_prices) // len(STOCKS)}")

# Check label distribution
print(f"\nLabel distribution (next-day return direction):")
print(df_prices['label'].value_counts(normalize=True))

Pulling stock prices from Yahoo Finance...
Date range: 2021-09-25 to 2022-10-05
  Fetching TSLA... 258 days
  Fetching TSM... 258 days
  Fetching AAPL... 258 days
  Fetching AMZN... 258 days
  Fetching MSFT... 258 days
  Fetching PG... 258 days
  Fetching NIO... 258 days
  Fetching META... 258 days
  Fetching AMD... 258 days
  Fetching NFLX... 258 days
  Fetching GOOG... 258 days

Total price records: 2838
Trading days per stock: ~258

Label distribution (next-day return direction):
label
0    0.519732
1    0.480268
Name: proportion, dtype: float64


**1.3 Aggregate tweets, merge with prices**

In [9]:
# Aggregate tweets by (stock, date)
print("\nAggregating tweets by stock and date...")

def aggregate_tweets(group):
    """Aggregate all tweets for a given stock-date pair"""
    return pd.Series({
        'text': ' '.join(group['tweet_clean'].values),
        'tweet_count': len(group),
        'avg_tweet_len': group['tweet_len'].mean(),
        'total_chars': group['tweet_len'].sum()
    })

df_agg = df.groupby(['Stock Name', 'date_only']).apply(aggregate_tweets).reset_index()
df_agg.columns = ['ticker', 'date_only', 'text', 'tweet_count', 'avg_tweet_len', 'total_chars']

print(f"Aggregated dataset shape: {df_agg.shape}")
print(f"Unique stock-date pairs: {len(df_agg)}")

# Merge with price data
print("\nMerging tweets with stock prices...")

df_merged = pd.merge(
    df_agg,
    df_prices,
    on=['ticker', 'date_only'],
    how='inner'
)

print(f"After merge: {len(df_merged)} samples")

# Remove rows with missing labels (last day has no next-day return)
before = len(df_merged)
df_merged = df_merged.dropna(subset=['label', 'next_day_return'])
df_merged['label'] = df_merged['label'].astype(int)
print(f"Removed {before - len(df_merged)} rows with missing labels")

# Final dataset summary
print("\n" + "=" * 50)
print("MERGED DATASET SUMMARY")
print("=" * 50)
print(f"Total samples: {len(df_merged)}")
print(f"Date range: {df_merged['date_only'].min()} to {df_merged['date_only'].max()}")
print(f"Unique trading days: {df_merged['date_only'].nunique()}")

print(f"\nSamples per stock:")
print(df_merged['ticker'].value_counts().sort_values(ascending=False))

print(f"\nLabel distribution:")
print(df_merged['label'].value_counts())
print(f"Class balance: {df_merged['label'].mean():.2%} positive (stock went up)")

print(f"\nTweet count stats (per stock-day):")
print(df_merged['tweet_count'].describe())

# Quick sanity check
print("\n" + "=" * 50)
print("SANITY CHECK: Sample rows")
print("=" * 50)
sample = df_merged[['ticker', 'date_only', 'tweet_count', 'next_day_return', 'label']].head(10)
print(sample.to_string(index=False))

# Save merged dataset
df_merged.to_parquet(f'{DATA_CLEAN}/merged_dataset.parquet', index=False)
print(f"\nSaved merged dataset to {DATA_CLEAN}/merged_dataset.parquet")


Aggregating tweets by stock and date...
Aggregated dataset shape: (3858, 6)
Unique stock-date pairs: 3858

Merging tweets with stock prices...
After merge: 2695 samples
Removed 0 rows with missing labels

MERGED DATASET SUMMARY
Total samples: 2695
Date range: 2021-09-30 to 2022-09-29
Unique trading days: 252

Samples per stock:
ticker
AAPL    252
AMZN    252
MSFT    252
TSM     252
TSLA    252
PG      252
NIO     252
AMD     249
GOOG    235
NFLX    228
META    219
Name: count, dtype: int64

Label distribution:
label
0    1401
1    1294
Name: count, dtype: int64
Class balance: 48.01% positive (stock went up)

Tweet count stats (per stock-day):
count    2695.000000
mean       22.256030
std        38.025532
min         1.000000
25%         5.000000
50%         9.000000
75%        17.000000
max       459.000000
Name: tweet_count, dtype: float64

SANITY CHECK: Sample rows
ticker  date_only  tweet_count  next_day_return  label
  AAPL 2021-09-30            7         0.008127      1
  AAPL 20

In [10]:
df_merged.head()

,ticker,date_only,text,tweet_count,avg_tweet_len,total_chars,close,return,next_day_return,label
0,AAPL,2021-09-30,"I need 1,000 shares of $AAPL Returns over the ...",7,169.0,1183,138.390686,-0.009312,0.008127,1
1,AAPL,2021-10-01,The Results! #LottoFriday $AAPL 141P | .65 to ...,10,197.6,1976,139.515427,0.008127,-0.024606,0
2,AAPL,2021-10-04,$FB $AAPL $AMZN $MSFT $GOOGL *APPLE STOCK FALL...,10,192.5,1925,136.082535,-0.024606,0.014158,1
3,AAPL,2021-10-05,10/05/21 - Watch today's #MarketOutlook video ...,5,173.8,869,138.009247,0.014158,0.006307,1
4,AAPL,2021-10-06,Buying $SHIB is like buying $AAPL before Steve...,10,175.5,1755,138.879700,0.006307,0.009084,1


**1.4 Train/val/test split**

In [11]:
# Sort by date
df_merged = df_merged.sort_values('date_only').reset_index(drop=True)

# Get unique dates and define split points
unique_dates = sorted(df_merged['date_only'].unique())
n_dates = len(unique_dates)

train_end_idx = int(n_dates * 0.70)
val_end_idx = int(n_dates * 0.85)

train_dates = unique_dates[:train_end_idx]
val_dates = unique_dates[train_end_idx:val_end_idx]
test_dates = unique_dates[val_end_idx:]

print("=" * 50)
print("TIME-BASED SPLIT")
print("=" * 50)
print(f"Total unique trading days: {n_dates}")
print(f"\nTrain: {len(train_dates)} days ({train_dates[0]} to {train_dates[-1]})")
print(f"Val:   {len(val_dates)} days ({val_dates[0]} to {val_dates[-1]})")
print(f"Test:  {len(test_dates)} days ({test_dates[0]} to {test_dates[-1]})")

# Create split masks
train_mask = df_merged['date_only'].isin(train_dates)
val_mask = df_merged['date_only'].isin(val_dates)
test_mask = df_merged['date_only'].isin(test_dates)

# Split the data
df_train = df_merged[train_mask].copy()
df_val = df_merged[val_mask].copy()
df_test = df_merged[test_mask].copy()

print(f"\nSamples:")
print(f"Train: {len(df_train)} ({len(df_train)/len(df_merged):.1%})")
print(f"Val:   {len(df_val)} ({len(df_val)/len(df_merged):.1%})")
print(f"Test:  {len(df_test)} ({len(df_test)/len(df_merged):.1%})")

# Check label distribution across splits
print(f"\nLabel distribution (% positive):")
print(f"Train: {df_train['label'].mean():.2%}")
print(f"Val:   {df_val['label'].mean():.2%}")
print(f"Test:  {df_test['label'].mean():.2%}")

# Save split indices
split_info = {
    'train_indices': df_train.index.values,
    'val_indices': df_val.index.values,
    'test_indices': df_test.index.values,
    'train_dates': [str(d) for d in train_dates],
    'val_dates': [str(d) for d in val_dates],
    'test_dates': [str(d) for d in test_dates]
}

np.savez(f'{DATA_CLEAN}/split_indices.npz',
         train_indices=split_info['train_indices'],
         val_indices=split_info['val_indices'],
         test_indices=split_info['test_indices'])

# Save date ranges as JSON for documentation
with open(f'{DATA_CLEAN}/split_dates.json', 'w') as f:
    json.dump({
        'train': {'start': str(train_dates[0]), 'end': str(train_dates[-1]), 'n_days': len(train_dates)},
        'val': {'start': str(val_dates[0]), 'end': str(val_dates[-1]), 'n_days': len(val_dates)},
        'test': {'start': str(test_dates[0]), 'end': str(test_dates[-1]), 'n_days': len(test_dates)}
    }, f, indent=2)

print(f"\nSaved split indices to {DATA_CLEAN}/split_indices.npz")
print(f"Saved split dates to {DATA_CLEAN}/split_dates.json")

TIME-BASED SPLIT
Total unique trading days: 252

Train: 176 days (2021-09-30 to 2022-06-10)
Val:   38 days (2022-06-13 to 2022-08-05)
Test:  38 days (2022-08-08 to 2022-09-29)

Samples:
Train: 1909 (70.8%)
Val:   395 (14.7%)
Test:  391 (14.5%)

Label distribution (% positive):
Train: 48.30%
Val:   56.20%
Test:  38.36%

Saved split indices to /content/drive/MyDrive/HODL Final Project/data_clean/split_indices.npz
Saved split dates to /content/drive/MyDrive/HODL Final Project/data_clean/split_dates.json


**1.5 Minimal feature engineering + baseline implementation**

In [12]:
# TF-IDF only
print("Fitting TF-IDF...")

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words='english')

X_train = tfidf.fit_transform(df_train['text'])
X_val = tfidf.transform(df_val['text'])
X_test = tfidf.transform(df_test['text'])

y_train = df_train['label'].values
y_val = df_val['label'].values
y_test = df_test['label'].values

print(f"TF-IDF shape: {X_train.shape}")

# Save vectorizer
with open(f'{OUTPUTS}/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# Evaluation function
def evaluate_model(y_true, y_pred, y_prob):
    """Standardized evaluation. Teammates use this."""
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'auc_roc': roc_auc_score(y_true, y_prob)
    }

# Naive baseline
naive_acc = max(y_test.mean(), 1 - y_test.mean())
print(f"\nNaive baseline (majority class): {naive_acc:.2%}")

# Logistic Regression
print("Training Logistic Regression...")

lr = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
y_prob = lr.predict_proba(X_test)[:, 1]

results = evaluate_model(y_test, y_pred, y_prob)

print(f"\n" + "=" * 50)
print("BASELINE RESULTS (Test Set)")
print("=" * 50)
print(f"Naive baseline: {naive_acc:.2%} accuracy")
print(f"Logistic Reg:   {results['accuracy']:.2%} accuracy, {results['f1']:.3f} F1, {results['auc_roc']:.3f} AUC-ROC")

# Save results
baseline_results = {
    'naive_accuracy': float(naive_acc),
    'logreg_accuracy': float(results['accuracy']),
    'logreg_f1': float(results['f1']),
    'logreg_auc': float(results['auc_roc'])
}

with open(f'{OUTPUTS}/baseline_results.json', 'w') as f:
    json.dump(baseline_results, f, indent=2)

print(f"\nSaved to {OUTPUTS}/baseline_results.json")

Fitting TF-IDF...
TF-IDF shape: (1909, 5000)

Naive baseline (majority class): 61.64%
Training Logistic Regression...

BASELINE RESULTS (Test Set)
Naive baseline: 61.64% accuracy
Logistic Reg:   50.90% accuracy, 0.389 F1, 0.497 AUC-ROC

Saved to /content/drive/MyDrive/HODL Final Project/outputs/baseline_results.json


**1.6 Export deliverables for team reference**

In [13]:
# Create eval_utils.py
eval_utils_code = '''
"""
Evaluation utilities for HODL Final Project.
All teammates should use these functions for consistent evaluation.
"""

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import pandas as pd

def evaluate_model(y_true, y_pred, y_prob):
    """
    Standardized evaluation function.

    Args:
        y_true: Ground truth labels (0 or 1)
        y_pred: Predicted labels (0 or 1)
        y_prob: Predicted probability of class 1

    Returns:
        dict with accuracy, f1, auc_roc
    """
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'auc_roc': roc_auc_score(y_true, y_prob)
    }

def print_results(results, model_name="Model"):
    """Pretty print results."""
    print(f"{model_name}: {results['accuracy']:.2%} acc, {results['f1']:.3f} F1, {results['auc_roc']:.3f} AUC")

def save_predictions(y_true, y_pred, y_prob, model_name, output_path):
    """Save predictions in standardized format."""
    df = pd.DataFrame({
        'y_true': y_true,
        'y_pred': y_pred,
        'y_prob': y_prob
    })
    df.to_csv(f'{output_path}/{model_name}_predictions.csv', index=False)
    print(f"Saved predictions to {output_path}/{model_name}_predictions.csv")
'''

with open(f'{OUTPUTS}/eval_utils.py', 'w') as f:
    f.write(eval_utils_code)

print("Created eval_utils.py")

# Create README
readme = f"""
# HODL Final Project

## Files You Need

- `data_clean/merged_dataset.parquet` - Dataset with text and labels
- `data_clean/split_indices.npz` - Train/val/test indices
- `outputs/eval_utils.py` - Evaluation function

## Usage
```python
import pandas as pd
import numpy as np
from eval_utils import evaluate_model

# Load data
df = pd.read_parquet('data_clean/merged_dataset.parquet')

# Load splits
splits = np.load('data_clean/split_indices.npz')
df_train = df.loc[splits['train_indices']]
df_val = df.loc[splits['val_indices']]
df_test = df.loc[splits['test_indices']]

# Text and labels
X_train, y_train = df_train['text'].values, df_train['label'].values
X_test, y_test = df_test['text'].values, df_test['label'].values

# ... train your model ...

# Evaluate (y_prob = predicted probability of class 1)
results = evaluate_model(y_test, y_pred, y_prob)
```

## Baseline to Beat (Test Set)

- Naive baseline: {naive_acc:.2%} accuracy
- Logistic Regression: {results['accuracy']:.2%} accuracy, {results['f1']:.3f} F1, {results['auc_roc']:.3f} AUC-ROC
"""

with open(f'{BASE_PATH}/README.md', 'w') as f:
    f.write(readme)

print("Created README.md")

# Summary of all exported files
print("\n" + "=" * 50)
print("EXPORT SUMMARY")
print("=" * 50)
print(f"\ndata_clean/")
print(f"  - merged_dataset.parquet")
print(f"  - split_indices.npz")
print(f"  - split_dates.json")
print(f"\noutputs/")
print(f"  - tfidf_vectorizer.pkl")
print(f"  - baseline_results.json")
print(f"  - eval_utils.py")
print(f"\nREADME.md")
print("\n" + "=" * 50)
print("NOTEBOOK 01 COMPLETE")
print("=" * 50)

Created eval_utils.py
Created README.md

EXPORT SUMMARY

data_clean/
  - merged_dataset.parquet
  - split_indices.npz
  - split_dates.json

outputs/
  - tfidf_vectorizer.pkl
  - baseline_results.json
  - eval_utils.py

README.md

NOTEBOOK 01 COMPLETE
